In [2]:
import pandas as pd
import time
import numpy as np
from sklearn.cross_validation import train_test_split
import lightgbm as lgb
import gc
import matplotlib.pyplot as plt
import os
%matplotlib inline
from sklearn.linear_model import LogisticRegression
from scipy.special import expit, logit
from sklearn.metrics import roc_auc_score

almost_zero = 1e-10
almost_one = 1 - almost_zero


traininfo = [['./sub_dart_train_BayesOpt_2018_0501_0125_11.csv', './sub_dart_train_BayesOpt_2018_0501_0315_38.csv',
              './sub_dart_train_BayesOpt_2018_0501_0518_48.csv', './sub_dart_train_BayesOpt_2018_0501_0715_06.csv'],
             ['./sub_xgdt_train_BayesOpt_2018_0430_1519_33.csv', './sub_xgdt_train_BayesOpt_2018_0430_1656_17.csv', 
              './sub_xgdt_train_BayesOpt_2018_0430_1844_31.csv', './sub_xgdt_train_BayesOpt_2018_0430_2026_42.csv']
            ]

validinfo = [['./sub_dart_valid_BayesOpt_2018_0501_0125_11.csv', './sub_dart_valid_BayesOpt_2018_0501_0315_38.csv',
              './sub_dart_valid_BayesOpt_2018_0501_0518_48.csv', './sub_dart_valid_BayesOpt_2018_0501_0715_06.csv'],
             ['./sub_xgdt_valid_BayesOpt_2018_0430_1519_33.csv', './sub_xgdt_valid_BayesOpt_2018_0430_1656_17.csv', 
              './sub_xgdt_valid_BayesOpt_2018_0430_1844_31.csv', './sub_xgdt_valid_BayesOpt_2018_0430_2026_42.csv']
            ]

testinfo = [['./sub_dart_BayesOpt_2018_0501_0148_24.csv', './sub_dart_BayesOpt_2018_0501_0339_24.csv',
             './sub_dart_BayesOpt_2018_0501_0544_49.csv', './sub_dart_BayesOpt_2018_0501_0739_43.csv'],
            ['./sub_it_BayesOpt_2018_0430_1527_02.csv', './sub_it_BayesOpt_2018_0430_1703_36.csv',
             './sub_it_BayesOpt_2018_0430_1854_57.csv', './sub_it_BayesOpt_2018_0430_2035_07.csv']]

train_trueinfo = [['./rawdata_train_From2400_2018_0501_0125_11.csv', './rawdata_train_From2400_2018_0501_0315_38.csv',
                   './rawdata_train_From2400_2018_0501_0518_48.csv', './rawdata_train_From2400_2018_0501_0715_06.csv'],
                  ['./rawdata_valid_From2400_2018_0501_0125_11.csv', './rawdata_valid_From2400_2018_0501_0315_38.csv',
                   './rawdata_valid_From2400_2018_0501_0518_48.csv', './rawdata_valid_From2400_2018_0501_0715_06.csv']
                 ]
                  
validlen = 5000000
models = []
data = pd.read_csv('./Dataset_xgdt_test__2018_0502_0903_41.csv', nrows=10)
data

,Unnamed: 0,device_nextClick,device_channel_nextClick,app_device_channel_nextClick,device_hour_nextClick,ip_channel_prevClick,ip_os_prevClick,nextClick,nextClick_shift,nextClick_shift_shift,...,ip_app_channel_mean_hour,X0,X1,X2,X3,X4,X5,X6,X7,X8
0,40000000,0.0,0.0,0.0,0.0,78231.0,909.0,1489713600,1.489785e+09,1.489785e+09,...,4.200000,66,2,6,22,16,1,43,251,4
1,40000001,0.0,0.0,0.0,0.0,71245.0,72494.0,399,1.489714e+09,1.489785e+09,...,8.027397,99,119,6,50,33,11,43,5118,20
2,40000002,0.0,0.0,0.0,0.0,71196.0,71872.0,88,3.990000e+02,1.489714e+09,...,8.295918,93,699,6,41,20,6,8,2764,30
3,40000003,0.0,1.0,1.0,0.0,77158.0,71344.0,425,8.800000e+01,3.990000e+02,...,3.800000,98,500,6,43,20,7,28,2450,33
4,40000004,0.0,0.0,0.0,0.0,8258.0,73446.0,1208,4.250000e+02,8.800000e+01,...,2.500000,72,82,6,27,16,3,31,252,20
5,40000005,0.0,0.0,0.0,0.0,78831.0,71077.0,56,1.208000e+03,4.250000e+02,...,6.895833,83,244,6,33,24,6,17,825,24
6,40000006,0.0,0.0,0.0,0.0,72543.0,73159.0,1489713600,5.600000e+01,1.208000e+03,...,5.000000,80,21,6,34,29,7,43,905,6
7,40000007,0.0,0.0,0.0,0.0,71613.0,71011.0,19251,1.489714e+09,5.600000e+01,...,6.000000,85,236,6,32,6,7,5,1280,25
8,40000008,0.0,0.0,0.0,0.0,71283.0,78583.0,1489713600,1.925100e+04,1.489714e+09,...,6.513889,87,19,7,33,26,4,17,1346,9
9,40000009,0.0,0.0,0.0,0.0,71545.0,74845.0,1489713600,1.489714e+09,1.925100e+04,...,8.500000,100,28,6,47,47,11,31,5492,14
